In [ ]:
import os

DATA_DIR = 'D:\DATASETS\Pokemon\large\images'
filepaths = []

for subdir in os.listdir(DATA_DIR):
    for file_name in os.listdir(os.path.join(DATA_DIR, subdir)):
        filepaths.append( os.path.join(DATA_DIR, subdir, file_name) )

In [ ]:

import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

TARGET_DIR = 'D:\\DATASETS\\Pokemon\\large\\256x256'
for i, fp in enumerate(filepaths):
    img = plt.imread(fp)
    resized = tf.image.resize(img, size=(256, 256)).numpy()
    resized = np.asarray(resized, dtype=np.uint8)
    save_path = os.path.join(TARGET_DIR, f'{i}.jpg')
    plt.imsave(save_path,resized)

In [13]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

TARGET_DIR = 'D:\\DATASETS\\Pokemon\\large\\256x256\\augmented'
DATA_DIR = 'D:\\DATASETS\\Pokemon\\large\\256x256\\resized'
filepaths = [os.path.join(DATA_DIR, x) for x in os.listdir(DATA_DIR)]

for i, fp in enumerate(filepaths):
    img_raw = plt.imread(fp)

    for j in range(1,5):    
        img = [img_raw]
        img = tf.keras.layers.RandomFlip(mode='horizontal')(img, training=True)
        img = tf.keras.layers.RandomRotation(factor=0.1, fill_mode='constant', fill_value=255.)(img, training=True)
        
        # augment colors
        full_white = 255*tf.ones_like(img)
        full_black = 0  *tf.ones_like(img)
        is_white = tf.stack(3*[tf.reduce_all((full_white-img<=20), axis=-1)], axis=-1)
        color_shift = tf.stack([tf.ones(img.shape[1:]) * tf.random.uniform(shape=[3], minval=0, maxval=255)
                                for _ in range(len(img))])
        img = (img + color_shift) % 255 # bring back into valid range
        img = tf.clip_by_value(img, 50, 200) # ensure 'natural' colors
        img = tf.where(is_white, full_white, img) # make white pixels white again

        img = np.asarray(img[0], dtype=np.uint8)
        plt.imsave(os.path.join(TARGET_DIR, f'{i}_a{j}.jpg'), img)   

In [9]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.celeb_a.load_data()
x_train = (x_train - (255/2)) / (255/2)

AttributeError: module 'keras.api._v2.keras.datasets' has no attribute 'celeb_a'

In [8]:
import tensorflow_datasets as tfds
tfds.()

SyntaxError: invalid syntax (3196769538.py, line 1)

In [4]:
None()

<>:1: SyntaxWarning: 'NoneType' object is not callable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'NoneType' object is not callable; perhaps you missed a comma?
C:\Users\Tom\AppData\Local\Temp\ipykernel_14356\744963307.py:1: SyntaxWarning: 'NoneType' object is not callable; perhaps you missed a comma?
  None()
C:\Users\Tom\AppData\Local\Temp\ipykernel_14356\744963307.py:1: SyntaxWarning: 'NoneType' object is not callable; perhaps you missed a comma?
  None()
C:\Users\Tom\AppData\Local\Temp\ipykernel_14356\744963307.py:1: SyntaxWarning: 'NoneType' object is not callable; perhaps you missed a comma?
  None()


TypeError: 'NoneType' object is not callable

In [ ]:
from mnist.architecture import generator, discriminator
from gan import GenerativeAdversarialNetwork
from utils import VisualizeGeneratedFakesCallback

generator.summary()
discriminator.summary()
z_shape = generator.input_shape[1:]
x_shape = discriminator.input_shape[1:]

print(x_shape)
print(z_shape)

model = GenerativeAdversarialNetwork(generator, discriminator, z_shape, x_shape, k=1)
model.compile()

# model.z_distribution.sample().shape

callbacks = [VisualizeGeneratedFakesCallback(z_distribution=model.z_distribution)]
model.fit(x=x_train, y=y_train, epochs=100, batch_size=64, callbacks=callbacks)

In [ ]:
import tensorflow as tf
import numpy as np
ce_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
p = np.exp(-1.5)
p_ = 1-p
print(p, p_)

logits = [1.5]
y_true = 0
loss = ce_loss([y_true] ,logits)

print(loss.numpy(), -y_true*np.log(p)-(1-y_true)*np.log(p_))

In [ ]:
import tensorflow as tf
import numpy as np
ce_loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
y_true = [0.]
y_pred = [0.7]

loss = ce_loss(y_true, y_pred) # -y_true * log(y_pred) - (1-y_true)*log(1-y_pred)
print(loss)

In [ ]:
import tensorflow as tf
import numpy as np
ce_loss_from_logits = tf.keras.losses.BinaryCrossentropy(from_logits=True)
ce_loss_from_probas = tf.keras.losses.BinaryCrossentropy(from_logits=False)

y_true = [0.]

y_pred = [2.] # logit
p1 = tf.keras.activations.sigmoid(tf.constant(y_pred))[0].numpy()
p2 = 1-p1
print(p1, p2)

loss_from_logits = ce_loss_from_logits(y_true, y_pred)
loss_from_probas = ce_loss_from_probas(y_true, p1)
loss_from_logits_2 = -y_true[0] * np.log(p1) - (1-y_true[0])*np.log(p2)
print(loss.numpy(), loss2)